<a href="https://colab.research.google.com/github/EGaraldi/EPIC_5/blob/main/Tutorials/tutorial_5/cosmo_sim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Analyzing cosmological simulations
==================================

# Step 0: getting the data
In a bit, we will use the data from the TNG100-3 simulation. Since they are a bit large, better start the download now:

In [1]:
!wget https://datashare.mpcdf.mpg.de/s/w7SqBPunS5dNSFE/download -O tng100-3.tar.gz
!mkdir tng100-3
!tar -xzvf tng100-3.tar.gz -C tng100-3
!rm tng100-3.tar.gz

--2025-08-04 23:50:34--  https://datashare.mpcdf.mpg.de/s/w7SqBPunS5dNSFE/download
Resolving datashare.mpcdf.mpg.de (datashare.mpcdf.mpg.de)... 130.183.207.3
Connecting to datashare.mpcdf.mpg.de (datashare.mpcdf.mpg.de)|130.183.207.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7935913673 (7.4G) [application/gzip]
Saving to: ‘tng100-3.tar.gz’

tng100-3.tar.gz     100%[===================>]   7.39G  25.8MB/s    in 5m 5s   

2025-08-04 23:55:40 (24.8 MB/s) - ‘tng100-3.tar.gz’ saved [7935913673/7935913673]



In [ ]:
# smaller simulation
!wget https://datashare.mpcdf.mpg.de/s/uYAfv1BYL3ffEf4/download -O small_sim_snap013.tar.gz
!mkdir small_sim
!tar -xzf small_sim_snap013.tar.gz -C small_sim
!rm small_sim_snap013.tar.gz

# What is a cosmological simulation?
In astrophysics, numerical simulations are a fundamental tool. There are two main reasons why:
* stars, planets, galaxies, ... are extremely complex systems, so even if we understand basic physical laws, it's impossible to predict how these interact with pen and paper
* science is based on a cycle of observation-hypothesis-experiments, but in astrophysics we can not manipulate the Universe to make experiments. Therefore, we need to turn to virtual universes (simulations) to make virtual experiments

These simulations combine different ingredients:
* gravity
* fluid dynamics
* chemistry
* (sometimes) radiation
* (sometimes) magnetic fields

Then, starting from our best guess on the conditions of the Universe after the Big Bang, we evolve these synthetic universes until today and use them to run experiments, understand observations, etc.

![sim](https://thesan-project.com/thesan/images/simulations_cartoon.png)

Let's start with some visualizations to get an idea of what we are dealing with. For example, the distribution of matter in one of these simulations looks like this:

![cosmic_web_millenium](https://astrobites.org/wp-content/uploads/2012/07/cosmic-web.jpg)

If we start to zoom in, we can see individual galaxies, for example here;

![galaxy](https://github.com/EGaraldi/EPIC_5/blob/main/Tutorials/tutorial_5/tng_visual.png?raw=1)

# The AREPO code
There are many different simulations codes, and each one of them works in a different way and saves the data in different formats.

For this tutorial, we will work with a simulation produced with a code called AREPO. Let's familiarize with it a bit

## Types of resolution elements

### Fliuds (gas)

AREPO uses a 3D mesh to discretize fluids. For example, the continuous fluid on the left, can be discretized using a Cartesian mesh (right) where each cell represents the fluid in it.  

![mesh](https://github.com/EGaraldi/EPIC_5/blob/main/Tutorials/tutorial_5/mesh.png?raw=1)

AREPO is unique because it uses a Voronoi mesh, a much more general and adaptive mesh with respect to the Cartesian one.

![voronoi](https://github.com/EGaraldi/EPIC_5/blob/main/Tutorials/tutorial_5/voronoi.png?raw=1)

Even better, this mesh adapts to the gas flow, so that it's resolution is better where there is more 'action'

![khi](https://github.com/EGaraldi/EPIC_5/blob/main/Tutorials/tutorial_5/khi_arepo.png?raw=1)
![gal](https://github.com/EGaraldi/EPIC_5/blob/main/Tutorials/tutorial_5/gal_arepo.png?raw=1)

Each one of the cells contains a single value for each of the quantity describing the fluid. For example, it might contain: density, velocity and temperature. But only one value for each of these is allowed in each cell.

### Dark Matter
Dark Matter is a mysterious substance that makes up 80% of the mass in the Universe. It seems to interact only through gravity (so, no light, no electro-magnetic fields, etc.). Because of this, it is NOT a fluid. AREPO represents it using particles. These are not fundamental particles, but rather represent large "pieces" of the Universe. Often these "pieces" are millions of time larger than the solar system, but still tiny compared to (most) galaxies. These particles have, by construction, all the same mass.

### Stars
Stars are also represented using particles. In reality, stars are large spheres of self-gravitating, nuclear burning gas. In fact, we can use CFD codes (including AREPO) to simulate them. But they are so tiny compared to a galaxy or the entire Universe, that they are often represented by particles that interact through gravity only. (Stellar collision are so incredibly rare that they do not matter at the scales of these simulations)

### Black hole, tracers, low-res particles, ...
There are many more "things" that can be simulated in AREPO. For today, we are going to ignore them.

## Solving equations

### gravity 
All components of the simulation interact through gravity. The gravitational force between two bodies 1 and 2 is:

$$F_{grav} = G \frac{M_1 \, M_2}{r_{12}^2}$$

If we have N bodies $i = 1, 2, 3, ..., N$, the number of force computations is: $N(N-1)/2$, i.e. the problem scales as $\mathcal{O}(N^2)$. This makes the simulation very slow!

To make it faster, cosmological simulations often use approximate methods. For example:
**particle-mesh**: 
* compute the mass density $\rho$ on a 3D grid (think of a 3D histogram of the particle mass)
* solve the Poisson equation on this grid to get the gravitational potential $\Phi$
$$ \nabla^2 \phi = 4\pi G \rho $$
* use $\Phi$ to compute $F_g = -m \nabla \Phi $
* very fast ($\mathcal{O}(N \log N)$), not very accurate close to particles because of resolution effects

**oct-tree**:
* arrange all particles in an oct-tree structure
![octtree](https://devforum-uploads.s3.dualstack.us-east-2.amazonaws.com/uploads/optimized/5X/e/c/4/3/ec4384970592733fa1afd8d3b5c726fd23d74e46_2_690x419.png)
* for distant nodes, use the full oct-tree node instead of the particles inside
$$ F_{grav} = \sum_{i \in node} G \frac{M_0 \, M_i}{r_{0i}^2} \approx G \frac{M_0 \, M_{node}}{r_{0node}^2}$$
* fast ($\mathcal{O}(N \log N)$), but less efficient than particle-mesh

AREPO uses a hybrid treePM algorithm, using oct-tree calculations at small scales and particle-mesh calculation at large distances.

### gas (fluid equations)

This approach is called _finite volumes_ approach, and is conceptually very different from the _finite differences_ method we saw yesterday. Why?

**Finite Differences**:
* Use grid points to approximate functions, derivatives, etc.
* Solves equations at the grid points.
* Best suited for structured (regular) grids.

**Finite Volumes**:
* Divides the domain into small control volumes (cells)
* Applies ("solves") conservation laws to each volume, tracking the flux of quantities across the boundaries of each cell.
* Naturally conserves quantities and works well with unstructured or adaptive meshes (like Voronoi meshes in AREPO).
* The solution is the average value within each cell, not at points.

In finite volume methods, we have to figure out if and how much of each quantity is transported across _each interface between cells_. This is called a Riemann problem. 

## Data format
AREPO outputs are in the [HDF5 format](https://www.hdfgroup.org/).

Like most simulations of structure formation, has two main types of outputs (but there can be many more):
* **snapshots**: full outputs storing the individual resolution elements of the simulation: (Voronoi) mesh, dark matter partcles, stellar particles, ...
* **groups catalog**: collection of structures found in the simulation


### Snapshot
A snapshot, in practice, is an HDF5 file with different HDF5 Groups. One group, called `Header` contains meta-data about the file itself and the simulation that produced it. Another one (`Config`) contains the setup of AREPO used for the simulation. Each other group contains the data concerning one type of resolution element. These groups are named `PartTypeN`, with `N` starting from 0 and increasing. The table below shows what each type `N` represents.
  | Type | Meaning     |
  | ---- | ----------- |
  | 0    | Gas         |
  | 1    | Dark Matter |
  | 4    | Stars       |
  | 5    | Black holes |

Each particle type has a number of fields associated, that changes for each type. We will see below how to find out which one are available.
- If all particles of one type have the same mass, this is not saved in the fields, but rather in the `MassTable` attribute of the `Header` group

### Group catalogs
The group catalogs contain two types of structures: Groups and Subhalos.
* **Groups or haloes** represent collections of particles that are close to each other, but there is no guarantee that they are physically associated.
* **Subhaloes** represent collections of particles that are physically bound to each other. We usually think of these as galaxies.
In both cases, the halo catalogs contain only the _summary properties_ of these collections. For example, the _total_ mass of the subhalo, the average temperature, etc.

### Units
Each field has its own units. They are defined in terms of the code _internal_ units, which are:
* Unit of length = $U_L$ = 1 kpc/$h$ = 3.08567e19 m/$h$
* Unit of Mass = $U_M$ = 1e10 Msun/$h$ = 1.98847e30 kg/$h$
* Unit of Velocity = $U_V$ = 1 km/s = 1e3 m/s

Here, $h=0.6774$ is something called _reduced Hubble constant_ and is a measure of how quickly the Universe expands. For today, we do not care about it.

Other units can be derived from these. So, for example, the unit of time is:

$$U_T = \frac{U_L}{U_V} = \frac{3.08567 \times 10^{19} \,m/h}{10^3\, m/s} = 3.08567 \times 10^{16} s/h = 9.78 \times 10^8 \, years/h$$

### File chunks
For efficiency reasons, a single snapshot or group catalog is often split into multiple _chunks_ that must be combined to obtain the full information. For today, the details are not important because we will use a library that does this for us.


### More info
- AREPO wiki (for public version only, but almost identical for the private version as well): https://gitlab.mpcdf.mpg.de/vrs/arepo/-/wikis/userguide/snapshotformat
- IllustrisTNG data specification: https://www.tng-project.org/data/docs/specifications/
- Thesan data specification: https://thesan-project.com/data.html

## Libraries
There are a few python libraries that can simplify the task of loading these data. For today's tutorial, we will use the [illustris_python](https://github.com/illustristng/illustris_python) library. You can install it in the following way:

In [2]:
!pip install --upgrade "git+https://github.com/illustristng/illustris_python.git"

  Cloning https://github.com/illustristng/illustris_python.git to /tmp/pip-req-build-iulrqrux
  Running command git clone --filter=blob:none --quiet https://github.com/illustristng/illustris_python.git /tmp/pip-req-build-iulrqrux
  Resolved https://github.com/illustristng/illustris_python.git to commit 2750c7d78270df78b3b3cd1c9e96eacc7512b59d
  Preparing metadata (setup.py) ... done
  Created wheel for illustris_python: filename=illustris_python-1.0.0-py3-none-any.whl size=15274 sha256=ac974de9aa81d53fe68a77d0d0661351b519317d529a46de376c9c11ab50058a
  Stored in directory: /tmp/pip-ephem-wheel-cache-9hik35wq/wheels/86/e4/00/01ba77a18a39023ada98456194fcef8898c3484b7136857a78
Successfully built illustris_python


## Additional resources
- AREPO wiki: https://gitlab.mpcdf.mpg.de/vrs/arepo/-/wikis/home
- code paper: https://arxiv.org/abs/0901.4107
- further development paper: https://arxiv.org/pdf/1503.00562
- RT solver paper: https://academic.oup.com/mnras/article/485/1/117/5303742
- introduction to the code (by me):  https://datashare.mpcdf.mpg.de/s/TmjvOoDm4Nm4Pnv

# Step 1: simple data exploration
Let's start by simply checking the data we have available.

In [3]:
import h5py
import numpy as np

print("SNAPSHOT FILE")
with h5py.File(f'tng100-3/output/snapdir_098/snap_098.0.hdf5', 'r') as snapfile:

    print("HDF5 groups available:")
    for group_name in snapfile.keys():
        print(f" - {group_name}")
    print()

    #print the attributes of the 'Header' group
    header = snapfile['Header']
    print("Attributes in the Header group:")
    for attr_name, attr_value in header.attrs.items():
        print(f" - {attr_name}: {attr_value}")
    print()

    #print the datasets available for each particle type:")
    for part_type in snapfile.keys():
        if part_type.startswith('PartType'):
            print(f"Particle type: {part_type}")
            part_data = snapfile[part_type]
            for dataset_name in part_data.keys():
                print(f" - {dataset_name}: shape {part_data[dataset_name].shape}, dtype {part_data[dataset_name].dtype}")
            print()

  # Let's do the same for the group catalogs
print("\nGROUP FILE")
with h5py.File(f'tng100-3/output/groups_098/fof_subhalo_tab_098.0.hdf5', 'r') as groupfile:

  print("HDF5 groups available:")
  for group_name in groupfile.keys():
      print(f" - {group_name}")
  print()

  #print the dataset available for groups and subhaloes
  print('Group datasets')
  for dataset_name in groupfile['Group'].keys():
      print(f" - {dataset_name}: shape {groupfile['Group'][dataset_name].shape}, dtype {groupfile['Group'][dataset_name].dtype}")
  print()

  print('Subhalo datasets')
  for dataset_name in groupfile['Subhalo'].keys():
      print(f" - {dataset_name}: shape {groupfile['Subhalo'][dataset_name].shape}, dtype {groupfile['Subhalo'][dataset_name].dtype}")

SNAPSHOT FILE
HDF5 groups available:
 - Config
 - Header
 - Parameters
 - PartType0
 - PartType1
 - PartType4
 - PartType5

Attributes in the Header group:
 - BoxSize: 75000.0
 - Composition_vector_length: 0
 - Flag_Cooling: 1
 - Flag_DoublePrecision: 0
 - Flag_Feedback: 1
 - Flag_Metals: 0
 - Flag_Sfr: 1
 - Flag_StellarAge: 0
 - Git_commit: b'd203ec8b07c7e2bdda5f608aa0babea46d603699'
 - Git_date: b'Thu Apr 7 14:14:27 2016 +0200'
 - HubbleParam: 0.6774
 - MassTable: [0.         0.03235675 0.         0.00302063 0.         0.        ]
 - NumFilesPerSnapshot: 7
 - NumPart_ThisFile: [12687804 13349540        0        0   329031     4397]
 - NumPart_Total: [88953586 94196375        0        0  2234312    30546]
 - NumPart_Total_HighWord: [0 0 0 0 0 0]
 - Omega0: 0.3089
 - OmegaBaryon: 0.0486
 - OmegaLambda: 0.6911
 - Redshift: 0.009521666967944764
 - Time: 0.99056814006128
 - UnitLength_in_cm: 3.085678e+21
 - UnitMass_in_g: 1.989e+43
 - UnitVelocity_in_cm_per_s: 100000.0

Particle type: Par

In [ ]:
with h5py.File('tng100-3/output/snapdir_098/snap_098.0.hdf5', 'r') as snapfile:
    print(snapfile['PartType0/Coordinates'][:10])

## File chunks and illustris_python
As mentioned, snapshot files are split in chunks, each one with (potentially) a different number of particles.

In [ ]:
with h5py.File('tng100-3/output/snapdir_098/snap_098.0.hdf5', 'r') as snapfile:
    Nfiles = snapfile['Header'].attrs['NumFilesPerSnapshot']

for ifile in range(Nfiles):
    with h5py.File(f'tng100-3/output/snapdir_098/snap_098.{ifile}.hdf5', 'r') as snapfile:
        print(f"File {ifile}:", snapfile['Header'].attrs['NumPart_ThisFile'])

So, if we want to load all gas particles, we have to explicitly loop over all files. For example:

In [ ]:
gas_positions = []

for ifile in range(Nfiles):
    with h5py.File(f'tng100-3/output/snapdir_098/snap_098.{ifile}.hdf5', 'r') as snapfile:
        gas_positions.append(snapfile['PartType0/Coordinates'][:])

Or, we can use the `illustris_python` to do this for us.

In [7]:
import illustris_python as il
il.snapshot.loadSubset

<function illustris_python.snapshot.loadSubset(basePath, snapNum, partType, fields=None, subset=None, mdi=None, sq=True, float32=False)>

In [ ]:
gas_data = il.snapshot.loadSubset('tng100-3/output', 98, 0, fields=['Coordinates'])

# Step 2: visualization
Let's start by checking how the simulation _looks like_. Naturally, the final results depends on what and how we plot. And naturally, this is not enough for scientific results. But it really helps to understand what is going on in the simulation.

For example, we can just plot all gas particles in the simulation..

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(8, 8))
ax.scatter(gas_data['Coordinates'][:, 0], gas_data['Coordinates'][:, 1], s=0.001, c='k', alpha=0.1)
ax.set_axis_off()
fig.show()

That's a mess! There's too many particles because we are projecting all particles along the $z$ direction! We can do better..

## Slice
A slice is simply a plot of all particles/grid cells/etc in a _slice_ through the simulation box, or one of their properties. The simplest thing we can do, is just to plot one dot per particle.

Try to adjust the code above to make a slice

In [ ]:
import matplotlib.pyplot as plt

# your code here

fig, ax = plt.subplots(figsize=(8, 8))
ax.scatter(gas_data['Coordinates'][slice_mask, 0], gas_data['Coordinates'][slice_mask, 1], s=0.001, c='k', alpha=0.1)
ax.set_axis_off()
fig.show()

We can see the **cosmic web**! But the dense parts are saturated because there are too many particles close to each other. We can fix this in few ways:
* play with point size and transparency
* make the slice thinner
* change how we plot this

**Exercise**: use a 2D histogram instead of `scatter` to make a map

In [ ]:
import matplotlib.pyplot as plt

min_z = 50000
max_z = 60000

slice_mask = (gas_data['Coordinates'][:, 2] >= min_z) & (gas_data['Coordinates'][:, 2] < max_z)

with h5py.File('tng100-3/output/snapdir_098/snap_098.0.hdf5', 'r') as snapfile:
    Lbox = snapfile['Header'].attrs['BoxSize']


fig, ax = plt.subplots(figsize=(8, 8))
h2d, _, _ = np.histogram2d(gas_data['Coordinates'][slice_mask, 0], gas_data['Coordinates'][slice_mask, 1], bins=300)
ax.imshow(np.log10(h2d.T + 1), origin='lower', cmap='viridis', extent=(0, Lbox, 0, Lbox))
ax.set_axis_off()
fig.show()

Now the color shows the gas density! We can see that there are clumps of matter (in yellow) at the nodes of this web. These are the haloes we were talking about earlier. To convince ourselves, let's mark them.

First, we can load the haloes in the simulation using:

In [ ]:
il.groupcat.loadHalos

In [ ]:
group_data = il.groupcat.loadHalos('tng100-3/output', 98, fields=['GroupPos'])
slice_mask_groups = (group_data['GroupPos'][:, 2] >= min_z) & (group_data['GroupPos'][:, 2] < max_z)


fig, ax = plt.subplots(figsize=(8, 8))
h2d, _, _ = np.histogram2d(gas_data['Coordinates'][slice_mask, 0], gas_data['Coordinates'][slice_mask, 1], bins=300)
ax.imshow(np.log10(h2d.T + 1), origin='lower', cmap='viridis', extent=(0, Lbox, 0, Lbox))

ax.scatter(group_data['GroupPos'][slice_mask_groups, 0], group_data['GroupPos'][slice_mask_groups, 1], marker='x', c='r', s=0.1)

ax.set_axis_off()
fig.show()

### Exercise: plot other physical quantities
Using a 2D histogram makes it easy to show different physical quantities. Try to plot different properties of the gas, or different types of particles (dark matter, stars, black holes)

**Hint**: to do this, you need to first load the gas properties. Have a look at the list of available properties from before, then load them alongside the positions, and use them for plotting.

### Periodic Boundary Conditions
Cosmological simulations often use PBC. This is the simplest way to simulate an 'infinite' Universe. I practice, it means that if a particle crosses one edge of the simulation, it "enters" from the opposite side.

This also means that we can shift the center of the particle distribution to any point we want. For example:

In [ ]:
def shift_center(positions, new_center):
    #your code here
    #hint: the new_center is a 3D vector, and the current center is at [0.5, 0.5, 0.5]* Lbox

In [ ]:
new_pos = shift_center(gas_data['Coordinates'], np.array([0.7, 0.3, 0.5])*Lbox)

slice_mask = (new_pos[:, 2] >= min_z) & (new_pos[:, 2] < max_z)

fig, ax = plt.subplots(figsize=(8, 8))
h2d, _, _ = np.histogram2d(new_pos[slice_mask, 0], new_pos[slice_mask, 1], bins=300, weights=gas_data['GFM_Metallicity'][slice_mask])
ax.imshow(np.log10(h2d.T + 1), origin='lower', cmap='inferno', extent=(0, Lbox, 0, Lbox))
ax.set_axis_off()
fig.show()

## Individual galaxies
We can try now to visualize individual galaxies. For this, we have to:
* identify where galaxies are, using the group catalogs
* load all and only the particles that belong to a galaxy, using group catalogs + snapshots
* visualize them

We can use illustris_python to do the first two steps for us.

In [ ]:
il.snapshot.loadSubhalo

In [ ]:
subhalo_id = 0

galaxy_data_gas   = il.snapshot.loadSubhalo('tng100-3/output', 98, subhalo_id, 0, fields=['Coordinates', 'Density', 'Velocities', 'GFM_Metallicity', 'InternalEnergy'])
galaxy_data_stars = il.snapshot.loadSubhalo('tng100-3/output', 98, subhalo_id, 4, fields=['Coordinates', 'GFM_StellarFormationTime'])

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
h2d, _, _ = np.histogram2d(galaxy_data_gas['Coordinates'][:, 0], galaxy_data_gas['Coordinates'][:, 1], bins=100)
ax.imshow(np.log10(h2d.T + 1), origin='lower', cmap='viridis', aspect='auto',
          extent=(galaxy_data_gas['Coordinates'][:, 0].min(), galaxy_data_gas['Coordinates'][:, 0].max(), galaxy_data_gas['Coordinates'][:, 1].min(), galaxy_data_gas['Coordinates'][:, 1].max()))
ax.scatter(galaxy_data_stars['Coordinates'][:, 0], galaxy_data_stars['Coordinates'][:, 1], marker='*', s=0.1, c='r')

ax.set_axis_off()
fig.show()

### Question: What is happening?

In [ ]:
#center the galaxy
subhalo_data = il.groupcat.loadSubhalos('tng100-3/output', 98, fields=['SubhaloPos', 'SubhaloMassInRadType', 'SubhaloHalfmassRadType'])
galaxy_center = subhalo_data['SubhaloPos'][subhalo_id]
galaxy_data_gas['Coordinates']   = shift_center(galaxy_data_gas['Coordinates'], galaxy_center)
galaxy_data_stars['Coordinates'] = shift_center(galaxy_data_stars['Coordinates'], galaxy_center)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
h2d, _, _ = np.histogram2d(galaxy_data_gas['Coordinates'][:, 0], galaxy_data_gas['Coordinates'][:, 1], bins=100)
ax.imshow(np.log10(h2d.T + 1), origin='lower', cmap='viridis', aspect='auto',
          extent=(galaxy_data_gas['Coordinates'][:, 0].min(), galaxy_data_gas['Coordinates'][:, 0].max(), galaxy_data_gas['Coordinates'][:, 1].min(), galaxy_data_gas['Coordinates'][:, 1].max()))
ax.scatter(galaxy_data_stars['Coordinates'][:, 0], galaxy_data_stars['Coordinates'][:, 1], marker='*', s=0.1, c='r')

ax.set_axis_off()
fig.show()

This shows us an important feture of galaxies. Stars are usually much more concentrated than gas! If you are not convinced, try changing the subhalo number and see!

## Stars
Now let's have a closer look at the stars.

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))

selection = np.full_like(galaxy_data_stars['Coordinates'][:, 0], True, dtype=bool) #all particles

h2d, _, _ = np.histogram2d(galaxy_data_stars['Coordinates'][selection, 0], galaxy_data_stars['Coordinates'][selection, 1], bins=100)
ax.imshow(np.log10(h2d.T + 1), origin='lower', cmap='cividis', aspect='auto',
          extent=(galaxy_data_stars['Coordinates'][selection, 0].min(), galaxy_data_stars['Coordinates'][selection, 0].max(), galaxy_data_stars['Coordinates'][selection, 1].min(), galaxy_data_stars['Coordinates'][selection, 1].max()))

ax.set_axis_off()
fig.show()

### Exercise: plot galaxy radius

We can define the radius of a galaxy using the `SubhaloHalfmassRadType` property of the subhalo. This gives us the radius that contains half the particles of each type. Since most of the time we see just the stars in the galaxy, we will use them to define a radius.

Your tasks:
* find the galaxy stellar radius
* plot only stars that are within this galaxy radius

In [ ]:
# your code here

### hot vs cold gas

The gas can have very different temperatures. Following the laws of thermodynamics, cold gas is dense while hot gas is rarefied.

Stars when gas collapses under its own gravity. Therefore, stars form in dense, cold gas. The energy they produce (radiation, winds, etc.) heats up the gas around them. This gas eventually cools down again and forms new stars.

AREPO does not save the gas temperature directly, but saves instead the internal (thermal) energy _per unit mass_, so we need a conversion to get the temperature:

In [ ]:
def computeParticlesTemperature(u, UnitVelocity_in_cm_per_s = 1e5, MeanMolecularWeight = 1, gamma = 5/3):
    """
    compute the temperature of the (gas) particles from their internal energy

    Parameters
    ----------

    u : numpy.array of float
        internal energy of the particles (as read from GADGET file). shape=(Npart)

    Returns
    -------
    temp : numpy.array of float
           temperature of the particles
    """

    BOLTZMANN = 1.3806e-16
    # units of u are energy/mass = (mass*length2/time2)/mass = (mass*length2/(length/velocity)2)/mass = velocity2
    PROTONMASS = 1.6726e-24  # g
    temp = MeanMolecularWeight*PROTONMASS/BOLTZMANN * (gamma-1) * u * UnitVelocity_in_cm_per_s**2

    return temp


In [ ]:
galaxy_data_gas['Temperature'] = computeParticlesTemperature(galaxy_data_gas['InternalEnergy'])

and we can finally plot the gas temperature:

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
h2d, _, _ = np.histogram2d(galaxy_data_gas['Coordinates'][:, 0], galaxy_data_gas['Coordinates'][:, 1], bins=100, weights=galaxy_data_gas['Density']*galaxy_data_gas['Temperature'])
ax.imshow(np.log10(h2d.T + 1), origin='lower', cmap='inferno', aspect='auto',
          extent=(galaxy_data_gas['Coordinates'][:, 0].min(), galaxy_data_gas['Coordinates'][:, 0].max(), galaxy_data_gas['Coordinates'][:, 1].min(), galaxy_data_gas['Coordinates'][:, 1].max()))

ax.set_axis_off()
fig.show()

### Exercise: plot hot and cold gas separately

For this, let's say that:
* cold gas = gas with $T \leq 10^6$ K
* hot gas = gas with $T > 10^6$ K

hint: use the temperature to select the gas to plot, then use a 2d histogram to show plot it

In [ ]:
#your code here

# Step 3: Phase-space diagram

Let's move to some quantitative analysis. A powerful tool is called **phase-space diagram**. This is the distribution of gas density and temperature in the simulation. From it, we can learn a lot about what the gas does.

In [ ]:
gas_data = il.snapshot.loadSubset('tng100-3/output', 98, 0, fields=['Density', 'InternalEnergy'])
gas_data['Temperature'] = computeParticlesTemperature(gas_data['InternalEnergy'])

In [ ]:
from matplotlib.colors import LogNorm

fig, ax = plt.subplots(figsize=(8, 8))
ax.hist2d(np.log10(gas_data['Density']), np.log10(gas_data['Temperature']), norm=LogNorm(), bins=100, cmap='viridis')

ax.set_xlabel(r'log$_{10}$(gas density)', fontsize=20)
ax.set_ylabel(r'log$_{10}$(gas temperature)', fontsize=20)

fig.show()

We can identify many different regions:
* equation of state gas
* hot diffuse gas produced by supernova
* warm gas produced by photo-ionisation
* the _baryon cycle_

# Step 4: Halo mass function
The halo mass function (HMF) is a basic characterization of how many haloes (collapsed structures) exist as a function of their mass.

The HMF is important for many reasons, including:

* Testing Cosmological Models: the HMF changes depending on the amount of dark and non-dark matter in the Universe, how fast it is expanding, _what is_ dark matter, etc. We can compare the predicted HMF with observations to understand the composition of the Universe.
* Structure Formation: halos are the building blocks of cosmic structure. Galaxies and clusters of galaxies form within these halos. Knowing the HMF allow us to better understand how galaxies and other visible objects formed after the Big Bang.

Specifically, the HMF is defined as the number _density_ of haloes in a given mass range:

$$HMF = \frac{dN(M_0 \leq M_\mathrm{halo} < M_1)}{dV \, d\log(M)}$$

where:
* $dN(M_0 \leq M_\mathrm{halo} < M_1)$ is the number of haloes with mass between $M_0$ and $M_1$,
* $dV$ is the volume used for counting the haloes and
* $d\log(M) = \log(M_1) - \log(M_0)$.

Here, however, we will use the HMF in a different way, i.e. to test the reliability of our simulation.

First of all, let's compute the HMF.

In [ ]:
halo_masses = group_data['Group_M_Mean200'] * 1e10 # M_sun/h

halo_masses = halo_masses[halo_masses > 0] #need to remove spurious structures

# Define bins of halo mass
Nbins = 50
log_masses = np.log10(halo_masses)
min_log_mass = log_masses.min()
max_log_mass = log_masses.max()
mass_bins = np.linspace(min_log_mass, max_log_mass, Nbins)


fig, ax = plt.subplots(figsize=(8, 6))

# count haloes of given mass and normalize
dN, _ = np.histogram(log_masses, bins=mass_bins)
dlogM = mass_bins[1] - mass_bins[0]
Lbox_in_cMpc = Lbox/1000
hmf = dN/dlogM/Lbox_in_cMpc**3

ax.step(mass_bins[:-1], np.log10(hmf), where='post', lw=3)

ax.set_xlabel(r'$\log(M_\mathrm{halo} \, [M_\odot/h])$')
ax.set_ylabel(r'$\log(dN/dlogM \, [h^3 \, cMpc^{-3}])$')
ax.set_title('Halo Mass Function')

fig.show()

Now let's compare this to some theoretical prediction. For this, we will use the `colossus` python package.

In [ ]:
!pip install colossus

In [ ]:
from colossus.cosmology import cosmology

#set the cosmology parameters
with h5py.File('tng100-3/output/snapdir_098/snap_098.0.hdf5', 'r') as snapfile:
    cosmo_params = {
        'flat': True,
        'H0': snapfile['Header'].attrs['HubbleParam'] * 100, #units km/s/Mpc
        'Om0': snapfile['Header'].attrs['Omega0'],
        'Ode0': snapfile['Header'].attrs['OmegaLambda'],
        'Ob0': snapfile['Header'].attrs['OmegaBaryon'],
        'sigma8': 0.8159,
        'ns': 0.9667
    }
    #for later
    redshift = snapfile['Header'].attrs['Redshift']

cosmology.setCosmology('myCosmo', cosmo_params)

In [ ]:
from colossus.lss import mass_function

# Colossus expects mass in Msun/h and computes dN/dln(M) -> we need to convert this to dN/dlog10(M)
bin_centers = (mass_bins[:-1] + mass_bins[1:]) * 0.5
jenkins_mf = mass_function.massFunction(10**bin_centers, redshift, mdef = 'fof', model = 'jenkins01', q_out = 'dndlnM')
jenkins_mf *= np.log(10)

angulo_mf = mass_function.massFunction(10**bin_centers, redshift, mdef = 'fof', model = 'angulo12', q_out = 'dndlnM')
angulo_mf *= np.log(10)

fig, ax = plt.subplots(figsize=(8, 6))

ax.step(mass_bins[:-1], np.log10(hmf), where='post', lw=3, label='TNG100-3')
ax.plot(bin_centers, np.log10(jenkins_mf), color='red', linestyle='--', lw = 3, label='Jenkins (2001)')
ax.plot(bin_centers, np.log10(angulo_mf), color='green', linestyle='--', lw = 3, label='Angulo (2012)')

ax.legend()
ax.set_xlabel('log(M_halo)')
ax.set_ylabel('log(dN/dlogM)')
ax.set_title('Halo Mass Function')

fig.show()

What is causing the big difference on the left side? To get a hint, let's mark the resolution of the simualation in the plot:

In [ ]:
with h5py.File('tng100-3/output/snapdir_098/snap_098.0.hdf5', 'r') as snapfile:
    particle_mass = snapfile['Header'].attrs['MassTable'][1] * 1e10 #DM particles mass

fig, ax = plt.subplots(figsize=(8, 6))

ax.step(mass_bins[:-1], np.log10(hmf), where='post', lw=3, label='TNG100-3')
ax.plot(bin_centers, np.log10(jenkins_mf), color='red', linestyle='--', lw = 3, label='Jenkins (2001)')
ax.plot(bin_centers, np.log10(angulo_mf), color='green', linestyle='--', lw = 3, label='Angulo (2012)')
ax.axvline(np.log10(particle_mass * 30), color='grey', linestyle=':', lw = 3, label='30 * DM particle mass')
ax.axvline(np.log10(particle_mass * 200), color='k', linestyle=':', lw = 3, label='200 * DM particle mass')

ax.legend()
ax.set_xlabel('log(M_halo)')
ax.set_ylabel('log(dN/dlogM)')
ax.set_title('Halo Mass Function')

fig.show()

We can see that the simulations starts to predict _less_ haloes than expected when these haloes have less than 200 particles, and really fails miserably for haloes that have less than 20. This happens because we have reached the **resolution limit** of the simulation.

At or below the resolution limit, the structures are sampled too coarsely to provide reliable results.
* In this particular case, what happens is that the potential well of the halo cannot be resolved by few particles, so its gravitational attraction is artificially suppressed. This prevents other nearby particles from falling into the halo, and the overall mass is under-estimated.

This also means that all haloes with less than 20 particles (or equivalent mass) are not well simulated, so we can not trust them. From now on, we will get rid of them!

# Step 5: Stellar-to-halo mass relation

The stellar-to-halo mass relation (SHMR) is one of the most important relations in the study of galaxy formation. It quantifies the efficiency with which galaxies convert the available material into stars.

The SHMR is determined by the interaction between many different physical processes:
* Gas cooling and accretion: how gas falls into the halo and cools to form stars.
* Star formation efficiency: how effectively gas is converted into stars.
* Feedback mechanisms: How much and in which form energy from stars and black holes is injected into the galaxies.

This means that any model that tries to explain the Universe and the formation of galaxies, must produce a realistic SHMR.

**exercise**: compute the SHMR for our simulation. Use only resolved haloes!

In [4]:
# your code here

## Exercise: understanding the SHMR

Now, we can try to understand what is the physics that produces the SHMR in pur simulation. A simple way to do so, is to color each point by some physical quantity and see if any of these quantities explains the SHMR.

Color the points in the plot with each one of the following quantities:
 - `SubhaloBHMass`
 - `SubhaloBHMdot`
 - `SubhaloGasMetallicity`
 - `SubhaloSFR`
 - sSFR = `SubhaloSFR`/M_star
 - `SubhaloSpin`
 - `SubhaloVel`


(hint: we have never loaded this quantities, so you will have to re-load the subhaloes and ask for these quantities as well)

(hint 2: some of these quantities span many orders of magnitude. If that's the case, try to use their logarithm to color the points)

In [ ]:
# your code here

# Step 6: Merger trees

Merger trees are structures that provide the history of groups/subhlaoes throughout the simulation. The main information they contain is whether and when two of such structures merged, but they also store basic properties of the halo though time.

In the file you downloaded, they are located in `tng100-3/postprocessing/trees/LHaloTree/`. As for the snapshot and groups, these are split in chunks. Moreover, inside each chunk there is a (large) number of HDF5 Groups called `TreeN`. These are basically just sub-chunks and we can ignore them for now.  Let's have a look at one of these tree.

In [5]:
with h5py.File('tng100-3/postprocessing/trees/LHaloTree/trees_sf1_099.0.hdf5', 'r') as treefile:
    print("Tree0 datasets:")
    for key in treefile['Tree0'].keys():
        print(f"- {key}")

Tree0 datasets:
- Descendant
- FileNr
- FirstHaloInFOFGroup
- FirstProgenitor
- GroupCM
- GroupMassType
- Group_M_Crit200
- Group_M_Crit500
- Group_M_Mean200
- Group_M_TopHat200
- Group_R_Crit200
- Group_R_Crit500
- Group_R_Mean200
- Group_R_TopHat200
- NextHaloInFOFGroup
- NextProgenitor
- SnapNum
- SubhaloBHMass
- SubhaloBHMdot
- SubhaloBfldDisk
- SubhaloBfldHalo
- SubhaloGasMetalFractions
- SubhaloGasMetalFractionsSfr
- SubhaloGasMetallicity
- SubhaloGasMetallicitySfr
- SubhaloGrNr
- SubhaloHalfmassRad
- SubhaloHalfmassRadType
- SubhaloIDMostBound
- SubhaloLen
- SubhaloLenType
- SubhaloMassInRadType
- SubhaloMassType
- SubhaloNumber
- SubhaloOffsetType
- SubhaloPos
- SubhaloSFR
- SubhaloSFRinRad
- SubhaloSpin
- SubhaloStarMetalFractions
- SubhaloStarMetallicity
- SubhaloStellarPhotometrics
- SubhaloVMax
- SubhaloVel
- SubhaloVelDisp


Some of these are copies of the subhalo properties saved in the group files, but now at all available simulation times in the same file.

Some other variables are new and they are used to define the connection between subhaloes progenitors/descendant (i.e. the subhalo at different snapshots). These are explained here:

![lhalotree](https://raw.githubusercontent.com/manodeep/LHaloTreeReader/refs/heads/master/lhalotree-mergertree-structure.png)

Some important remarks:
* `SubhaloNumber` is an identifier assigned to each subhalo. It is **unique** within any individual snapshot, but not across different snapshot. (i.e. we can not have two subhaloes with number 7 at the same snapshot, but we can have a subhalo 7 in snapshot 98 and another subhalo 7 in snapshot 99, and they are NOT related to each other.
* `Descendant`, `FirstProgenitor` and `NextProgenitor` are **indexes** to the arrays in the merger tree. So if we have a halo at _position_ `p=0`, its first progenitor is at _position_ `FirstProgenitor[p]`, and the identifier of the progenitor is `SubhaloNumber[FirstProgenitor[p]]`.
* To find the non-first progenitors of a halo, we neet to first find its `FirstProgenitor` and then walk to the `NextProgenitor` of it, and again, and again, ... until we run out of `NextProgenitor` (i.e. we find `NextProgenitor = -1`.

The trees are usually walked backwards: we start from a halo, and load all its history at previous time in the simulation, including the histories of all the haloes that merged to form it.
* For example, in the picture above, we can select the central halo in the bottom row and follow all its progenitors in previous snapshots: the central 3 subhalo in the penultimate row, 4 of them in the row above, etc.

We can use `illustris_python` to load the past tree of a specific subhalo using:

In [ ]:
il.lhalotree.loadTree

We can pass the `onlyMPB=True` argument to only load the **main progenitor branch**. What this mean is: whenever a halo has multiple progenitors, we only follow the _main_ progenitor, i.e. the most massive one.

In [26]:
tree0 = il.lhalotree.loadTree('tng100-3/output', 98, 0, onlyMPB=True)

In [ ]:
fig, ax = plt.subplots(1,1)

ax.plot(tree0['SnapNum'], np.log10(tree0['SubhaloMassType'][:,1] * 1e10)) #convert mass to Msun/h
ax.set_xlabel('Output number')
ax.set_ylabel(r'log( Subhalo Mass [M$_\odot$/h] )')

We can now load the entire history of the subhalo by using `onlyMPB=False` and remake the plot.

**Important**: when we use `illustris_python` to load the merger tree, we lose all the indexing structures, because we are loading only a small portion of the full tree structure, so indexes do not make sense anymore.
* What `illustris_python` does, is to give us an array where each branch of the merger tree is placed one after the other.
* There is a (complicated) rule for ordering them, but for today we just need to know that a halo history is all in a contiguous piece of the array(s) returned, and stops when it merges to the main branch.

In [14]:
tree0 = il.lhalotree.loadTree('tng100-3/output', 98, 0, onlyMPB=False)

In [ ]:
fig, ax = plt.subplots(1,1)

ax.plot(tree0['SnapNum'], np.log10(tree0['SubhaloMassType'][:,1] * 1e10)) #convert mass to Msun/h
ax.set_xlabel('Output number')
ax.set_ylabel(r'log( Subhalo Mass [M$_\odot$/h] )')

**Question**: What do you see? Why is that?

## identify mergers
The next task we have is to identify mergers, i.e. when two or more subhalo merge together to form a larger structure.

In [ ]:
def find_mergers(tree):
    mergers_snap = []
    mergers_haloes = []
    for snap in np.unique(tree['SnapNum']):
        snap_mask = tree['SnapNum'] == snap
        for desc in np.unique(tree['Descendant'][snap_mask]):
            desc_mask = tree['Descendant'][snap_mask] == desc
            if np.where(desc_mask)[0].size > 1:
                mergers_snap.append(snap)
                mergers_haloes.append(tree['SubhaloNumber'][snap_mask][desc_mask])
    return mergers_snap, mergers_haloes

In [ ]:
mergers_snap, mergers_haloes = find_mergers(tree0)

In [ ]:
for s, id in zip(mergers_snap, mergers_haloes):
    print(f"merger at snap {s} between subhaloes {id}")

In [21]:
treeN = il.lhalotree.loadTree('tng100-3/output', 98, 1455, onlyMPB=False)

In [ ]:
fig, ax = plt.subplots(1,1)

ax.plot(treeN['SnapNum'], np.log10(treeN['SubhaloMassType'][:,1] * 1e10)) #convert mass to Msun/h
ax.set_xlabel('Output number')
ax.set_ylabel(r'log( Subhalo Mass [M$_\odot$/h] )')

In [ ]:
mergers_snap, mergers_haloes = find_mergers(treeN)

In [ ]:
fig, ax = plt.subplots(1,1)

breaks = np.where(treeN['SnapNum'][:-1] - treeN['SnapNum'][1:] < 0)[0]

#isolate main branch
main_branch = {'SnapNum': treeN['SnapNum'][:breaks[0]+1], 'SubhaloMassType':treeN['SubhaloMassType'][:breaks[0]+1,:]}

#plot main branch
ax.plot(main_branch['SnapNum'], np.log10(main_branch['SubhaloMassType'][:,1] * 1e10), 'k-', lw=4) #convert mass to Msun/h


#plot other branches
current_idx = breaks[0]+1
for i,b in enumerate(breaks[1:]):
    color = f'C{i%8}'
    ax.plot(treeN['SnapNum'][current_idx:b+1], np.log10(treeN['SubhaloMassType'][current_idx:b+1,1] * 1e10), '-', color=color) #convert mass to Msun/h
    #mark merger
    mbidx = np.where(main_branch['SnapNum'] == treeN['SnapNum'][current_idx])[0][0]
    plt.plot([main_branch['SnapNum'][mbidx], main_branch['SnapNum'][mbidx]], [np.log10(treeN['SubhaloMassType'][current_idx,1] * 1e10), np.log10(main_branch['SubhaloMassType'][mbidx,1] * 1e10)], ls='--', color=color) #dashed line
    ax.plot(main_branch['SnapNum'][mbidx], np.log10(main_branch['SubhaloMassType'][mbidx,1] * 1e10), '*', color='yellow', mec='goldenrod', markersize=10) # star

    current_idx = b+1

color = f'C{i+1%8}'
ax.plot(treeN['SnapNum'][current_idx:], np.log10(treeN['SubhaloMassType'][current_idx:,1] * 1e10), '-', color=color) #convert mass to Msun/h
#mark merger
mbidx = np.where(main_branch['SnapNum'] == treeN['SnapNum'][current_idx])[0][0]
plt.plot([main_branch['SnapNum'][mbidx], main_branch['SnapNum'][mbidx]], [np.log10(treeN['SubhaloMassType'][current_idx,1] * 1e10), np.log10(main_branch['SubhaloMassType'][mbidx,1] * 1e10)], ls='--', color=color) #dashed line
ax.plot(main_branch['SnapNum'][mbidx]                           , np.log10(main_branch['SubhaloMassType'][mbidx,1] * 1e10), '*', color='yellow', mec='goldenrod', markersize=10) # star

ax.set_xlabel('Output number')
ax.set_ylabel(r'log( Subhalo Mass [M$_\odot$/h] )')

# Extra

Notebook with solutions: <a href="https://colab.research.google.com/github/EGaraldi/EPIC_5/blob/main/Tutorials/tutorial_5/cosmo_sim_with_solutions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>